# Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import KFold

pd.set_option('display.max_columns', 500)

# Read in and preprocess Data

In [2]:
# read in data
train_features = pd.read_csv("train_features.csv")
train_labels = pd.read_csv("train_labels.csv")
test_features = pd.read_csv("test_features.csv")

pid_lst = set(list(test_features.iloc[:, 0]))

In [3]:
train_features = train_features.groupby('pid', group_keys = False).mean()
test_features = test_features.groupby('pid', group_keys = False).mean()
train_labels = train_labels.sort_values('pid')

train_labels = train_labels.set_index('pid')


# Subtask 1

In [8]:
##BEST PARAMETERS
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 125, 'nthread': 1, 'objective': 'binary:logistic', 'reg_lambda': 1.5, 'seed': 33, 'silent': 1, 'subsample': 0.8, 'alpha' : 12}
# xgb_params = {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 125, 'nthread': 1, 'objective': 'binary:logistic', 'reg_lambda': 1.5, 'seed': 33, 'silent': 1, 'subsample': 0.8, 'alpha' : 12}

results = []

labels = ["LABEL_BaseExcess", "LABEL_Fibrinogen", "LABEL_AST", 
          "LABEL_Alkalinephos", "LABEL_Bilirubin_total", "LABEL_Lactate", 
          "LABEL_TroponinI", "LABEL_SaO2",  "LABEL_Bilirubin_direct", "LABEL_EtCO2"]
 
for i in range(len(labels)):
    mod = xgb.XGBClassifier()
    mod.fit(train_features, train_labels.loc[:, labels].iloc[:,i])
    y_pred = mod.predict_proba(test_features)
    results.append(y_pred[:,1])
    
    

C:\Users\Damja\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:10:56] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:10:57] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:10:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:11:01] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [10]:
results_df = pd.DataFrame(results)
results_df = results_df.transpose()
results_df.columns = labels
results_df

,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2
0,0.992289,0.168586,0.855396,0.759530,0.886765,0.330509,0.000088,0.142081,0.054636,0.000308
1,0.053830,0.034151,0.454847,0.354147,0.424002,0.049693,0.055556,0.106257,0.009528,0.040849
2,0.037889,0.057389,0.119253,0.119768,0.137706,0.052663,0.051922,0.030176,0.004817,0.001938
3,0.988467,0.984169,0.967692,0.970338,0.966823,0.501926,0.001836,0.786200,0.780134,0.000361
4,0.056855,0.004896,0.130194,0.080578,0.061907,0.031172,0.025853,0.025814,0.001265,0.000115
...,...,...,...,...,...,...,...,...,...,...
12659,0.020784,0.035085,0.109951,0.099931,0.072070,0.070524,0.001691,0.023081,0.005224,0.000024
12660,0.718012,0.024549,0.614295,0.389916,0.392129,0.352833,0.003531,0.196376,0.001085,0.000451
12661,0.932557,0.001531,0.066884,0.106087,0.057220,0.145873,0.000568,0.560915,0.011087,0.000075
12662,0.000937,0.001848,0.236114,0.294968,0.070343,0.019834,0.241650,0.030725,0.007430,0.003068


# Subtask 2

In [12]:
#lets try xgboost again
##BEST PARAMETERS
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 125, 'nthread': 1, 'objective': 'binary:logistic', 'reg_lambda': 1.5, 'seed': 33, 'silent': 1, 'subsample': 0.8, 'alpha' : 12}
# xgb_params = {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 11, 'missing': -999, 'n_estimators': 125, 'nthread': 1, 'objective': 'binary:logistic', 'reg_lambda': 1.5, 'seed': 33, 'silent': 1, 'subsample': 0.8, 'alpha' : 12}

#colsample_bytree = 0.7, learning_rate = 0.05, max_depth = 30, in_child_weight = 11, n_estimators = 12, nthread = 1, objective = "binary:logistic", reg_lambda = 1.5, seed = 33, silent = 1, subsample = 0.8)
#scale_pos_weight = XY, due to imbalancededness

results = []
labels = ["LABEL_Sepsis"]

mod = xgb.XGBClassifier()
mod.fit(train_features, train_labels.loc[:, labels])
y_pred = mod.predict_proba(test_features)
results.append(y_pred[:,1])


C:\Users\Damja\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Damja\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[14:11:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [14]:
results_df2 = pd.DataFrame(results)
results_df2 = results_df2.transpose()
results_df2.columns = labels

(results_df2 ) 



,LABEL_Sepsis
0,0.028821
1,0.024111
2,0.011090
3,0.008528
4,0.016754
...,...
12659,0.009826
12660,0.000992
12661,0.031564
12662,0.011238


# Subtask 3

In [15]:
results = []

mod = xgb.XGBRegressor()
labels = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

for i in range(len(labels)):
    mod = xgb.XGBRegressor()
    mod.fit(train_features, train_labels.loc[:, labels].iloc[:,i])
    y_pred = mod.predict(test_features)
    results.append(y_pred)

In [17]:
results_df3 = pd.DataFrame(results)
results_df3 = results_df3.transpose()
results_df3.columns = labels

# Save Data

In [18]:
pid_lst = pd.DataFrame(pid_lst)  

In [20]:
### USE PID LIST!

labels_all = ["pid", "LABEL_BaseExcess", "LABEL_Fibrinogen",
              "LABEL_AST", "LABEL_Alkalinephos", "LABEL_Bilirubin_total",
              "LABEL_Lactate", "LABEL_TroponinI", "LABEL_SaO2", 
              "LABEL_Bilirubin_direct", "LABEL_EtCO2", "LABEL_Sepsis",
             'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']


preds = pd.DataFrame(np.column_stack((pid_lst, results_df, results_df2, results_df3)))

preds.columns = labels_all

preds
# same format as submit file
#pd.read_csv("sample.csv")

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0.0,0.992289,0.168586,0.855396,0.759530,0.886765,0.330509,0.000088,0.142081,0.054636,0.000308,0.028821,15.058461,87.189697,98.254822,81.866043
1,3.0,0.053830,0.034151,0.454847,0.354147,0.424002,0.049693,0.055556,0.106257,0.009528,0.040849,0.024111,17.443316,82.117645,96.654160,100.327919
2,5.0,0.037889,0.057389,0.119253,0.119768,0.137706,0.052663,0.051922,0.030176,0.004817,0.001938,0.011090,18.491133,71.692032,95.548073,69.732536
3,7.0,0.988467,0.984169,0.967692,0.970338,0.966823,0.501926,0.001836,0.786200,0.780134,0.000361,0.008528,16.586477,84.039352,97.696236,89.786736
4,9.0,0.056855,0.004896,0.130194,0.080578,0.061907,0.031172,0.025853,0.025814,0.001265,0.000115,0.016754,19.623606,90.662109,96.716347,91.523109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,31647.0,0.020784,0.035085,0.109951,0.099931,0.072070,0.070524,0.001691,0.023081,0.005224,0.000024,0.009826,15.941040,68.953674,96.986725,69.888908
12660,31649.0,0.718012,0.024549,0.614295,0.389916,0.392129,0.352833,0.003531,0.196376,0.001085,0.000451,0.000992,15.210969,81.251015,98.018600,89.168579
12661,31651.0,0.932557,0.001531,0.066884,0.106087,0.057220,0.145873,0.000568,0.560915,0.011087,0.000075,0.031564,19.263367,76.438370,98.578186,92.176033
12662,31652.0,0.000937,0.001848,0.236114,0.294968,0.070343,0.019834,0.241650,0.030725,0.007430,0.003068,0.011238,19.523912,95.555946,97.838211,113.859863


In [21]:
# preds is a pandas dataframe containing the result
preds.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')